In [0]:
!pip install tweepy

In [0]:
import tweepy
import pandas as pd

#Add your credentials here
twitter_keys = {
        'consumer_key':        'XXX',
        'consumer_secret':     'XXX',
        'access_token_key':    'XXX',
        'access_token_secret': 'XXX'
    }

In [0]:
#Setup access to API
auth = tweepy.OAuthHandler(twitter_keys['consumer_key'], twitter_keys['consumer_secret'])
auth.set_access_token(twitter_keys['access_token_key'], twitter_keys['access_token_secret'])

api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [0]:
tweets = []
count = 1
from dateutil.parser import parse
from datetime import datetime, timedelta
from pytz import timezone
 
gmt = timezone('Asia/Kolkata')
last_hour_date_time = datetime.now(gmt) - timedelta(hours = 150)
print(last_hour_date_time)

users = []
page_count = 0

for tweet in tweepy.Cursor(api.search, q="#covid OR #covid19 OR #covid-19 OR #corona OR #StayHome OR #StaySafe OR #coronavirus -filter:retweets", 
                           count=5000,  since=last_hour_date_time.strftime('%Y-%m-%d %H'), tweet_mode='extended', lang="en").items(1000):

#for tweet in tweepy.Cursor(api.search, q="#covid OR #covid19 OR #covid-19 OR #corona OR #StayHome OR #StaySafe OR #coronavirus -filter:retweets", 
#                           count=5000, since='2020-04-06', tweet_mode='extended', lang="en").items(1000):
    data = [tweet.created_at, tweet.full_text, tweet.user._json['name'], tweet.user._json['screen_name'], tweet.user._json['id'], tweet.user._json['location'], 
            tweet.user._json['followers_count'], parse(tweet.user._json['created_at']).strftime("%d-%m-%Y"), tweet.user._json['statuses_count'], tweet.user._json['friends_count'],
            tweet.user._json['listed_count'], tweet.user._json['favourites_count']]

    data = tuple(data)
    tweets.append(data)
    continue
	
df = pd.DataFrame(tweets, columns = ['tweet_time', 'tweet_text', 'username', 'user_screenname', 'userid', 'location', 'followers_count',
                                     'twtr_joined_on', 'statuses_count', 'friends_count', 'listed_count', 'favourites_count'])

print("df ready : ", df.shape )

In [0]:
#Preprocessing
import nltk
nltk.download('punkt')
nltk.download('stopwords')

!pip install tweet-preprocessor
import preprocessor as p
p.set_options(p.OPT.URL, p.OPT.MENTION, p.OPT.SMILEY, p.OPT.EMOJI)

In [0]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
cleantweetsNOPunct = []
import string
table = str.maketrans('','', string.punctuation)

for tweet in df['tweet_text']:
    try:
        tweet = p.clean(tweet)
        #tokenize
        words2 = word_tokenize(tweet.lower())
        #remove puncts
        words3 = [w.translate(table) for w in words2]
        cleantweetsNOPunct.append((" ".join(words3)).strip())
    except:
        cleantweetsNOPunct.append(tweet)
        continue

print(len(cleantweetsNOPunct))

df['clean_Tweet_NoPunct'] = cleantweetsNOPunct

In [0]:
#filename = "Covid19_Tweets" + str(datetime.now(gmt).strftime('%Y-%m-%d %H:%M')) + ".csv"
filename = "Covid19_Tweets" + str(last_hour_date_time).replace(" ", "") + ".csv"

df.to_csv(filename, index=False)
print(filename + " stored as csv")